## Initialization

In [ ]:
# Imports
import os
import sys
import glob
import shutil

import pandas as pd
from sklearn.metrics import mutual_info_score

# Add repository root to PYTHONPATH for local imports
PROJECT_ROOT = os.path.abspath(os.getcwd())
sys.path.insert(0, PROJECT_ROOT)

# Columns excluded from uniqueness counts
exclude = ["Label", "Case_ID"]

In [ ]:
def find_optimal_bins(series, target, k_min=2, k_max=10):
    """
    For a numeric pandas Series and a discrete target array/Series,
    compute mutual information for each k in [k_min..k_max]
    using equal-frequency bins, and return the best k.
    """
    mi_scores = {}
    # drop NaNs in both series & align
    valid = series.notna() & pd.Series(target).notna()
    x = series[valid]
    y = pd.Series(target)[valid].values

    for k in range(k_min, k_max + 1):
        try:
            # qcut may drop bins if too many duplicates → use 'duplicates="drop"'
            bins = pd.qcut(x, q=k, duplicates="drop")
        except ValueError:
            # e.g., not enough unique values to form k bins
            continue

        # codes: 0..(n_bins-1)
        codes = bins.cat.codes
        # compute MI
        mi = mutual_info_score(y, codes)
        mi_scores[k] = mi

    if not mi_scores:
        return None, {}
    # pick k with highest MI
    best_k = max(mi_scores, key=mi_scores.get)
    return best_k, mi_scores

## Traffic

### Load each df

In [ ]:
feature_folders = {
    'decl3': 'traffic_decl3_features',
    'mr_tr': 'traffic_mr_tr_features',
    'payload': 'traffic_payload_Pay36_features'
}

# Initialize dict for all DataFrames
dataframes = {}

# Remove all files in each subfolder except the one named <subfolder>.csv
for prefix, folder_name in feature_folders.items():
    base_dir = os.path.join(
        PROJECT_ROOT,
        '3.1_selected_features',
        'traffic',
        folder_name
    )
    # Loop over each subfolder (e.g. 'mr', 'dc_data', etc.)
    for subfolder in os.listdir(base_dir):
        subfolder_dir = os.path.join(base_dir, subfolder)
        if not os.path.isdir(subfolder_dir):
            continue

        # The only file we want to keep
        keep_name = f"{subfolder}.csv"

        # Delete everything else in this folder
        for fname in os.listdir(subfolder_dir):
            fpath = os.path.join(subfolder_dir, fname)
            if os.path.isfile(fpath) and fname != keep_name:
                os.remove(fpath)
                print(f"Removed unwanted file: {fpath}")

    # Find all CSV files one level down (e.g. bs_data/bs_data.csv, dc_data/dc_data.csv, etc.)
    csv_paths = glob.glob(os.path.join(base_dir, '*', '*.csv'))
    
    for path in csv_paths:
        # Extract the subfolder name, e.g. 'bs_data'
        subfolder = os.path.basename(os.path.dirname(path))
        # Create a prefixed key for the DataFrame dict
        key = f"{prefix}_{subfolder}"
        # Read the CSV
        df = pd.read_csv(path)
        dataframes[key] = df
        print(f"Loaded {key!r}: shape={df.shape}")

# Example access:
# dataframes['decl3_bs_data'], dataframes['mr_tr_dc_data'], dataframes['payload_baseline'], etc.


### Show unique values per df

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

def show_unique_counts(name):
    if name not in unique_counts_dict:
        print(f"No dataset named {name!r}. Available: {list(dataframes.keys())}")
        return
    counts = unique_counts_dict[name]
    total  = total_unique_dict[name]
    print(f"Unique values per column in {name!r}:")
    print(counts)
    print(f"\nTotal unique values (excluding {exclude}): {total}")

In [ ]:
show_unique_counts('mr_tr_bs_data')

### Strategy 1: Look for the optimal number of bins for variables with +50 unique values

In [ ]:
# threshold_unique = 50
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=10)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl3_bs_data':")
# for col, info in all_results['decl3_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Strategy 2: Hard coding 2/3 bins and a threshold of 10+ unique values

In [ ]:
threshold_unique = 10
all_results = {}

for name, df in dataframes.items():
    # skip any dataset without the target
    if 'Label' not in df.columns:
        print(f"Skipping {name!r}: no 'Label' column")
        continue

    results = {}
    for col in df.columns:
        # you can also skip Label and Case_ID explicitly
        if col in ('Label','Case_ID'):
            continue
        if not pd.api.types.is_numeric_dtype(df[col]):
            continue
        if df[col].nunique() <= threshold_unique:
            continue

        best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
        results[col] = {'best_k': best_k, 'mi_by_k': scores}

    all_results[name] = results

In [ ]:
# Example: inspect bs_data results
print("Optimal bins for 'decl3_bs_data':")
for col, info in all_results['decl3_bs_data'].items():
    print(f" • {col}: {info['best_k']} bins")

In [ ]:
for name, results in all_results.items():
    df = dataframes[name]
    binned_from = []
    for col, info in results.items():
        k = info['best_k']
        # skip features where we couldn’t find a valid k
        if k is None:
            continue

        # apply equal-frequency bins with qcut
        binned_col = f"{col}_binned"
        df[binned_col] = pd.qcut(
            df[col],
            q=k,
            duplicates="drop"
        )
        binned_from.append(col)
        print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

    # drop all the originals at once (ignore any that might be missing)
    df = df.drop(columns=binned_from, errors='ignore')

    # save back if you want to overwrite
    dataframes[name] = df

### Strategy 3: Hard coding 2/3 bins and a threshold of 3+ unique values

In [ ]:
# threshold_unique = 3
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl3_bs_data':")
# for col, info in all_results['decl3_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Show unique values per df after binning

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

In [ ]:
show_unique_counts('decl3_bs_data')

### Export to 3.1_binned_logs

In [ ]:
# Define output root (parallel to '3_processed_logs')
output_root = os.path.join(PROJECT_ROOT, '3.2_binned_features', 'traffic')

# Save each DataFrame to its mirrored folder structure
for key, df in dataframes.items():
    # Determine which prefix this key uses
    prefix = None
    for p in feature_folders:
        if key.startswith(f"{p}_"):
            prefix = p
            break
    if prefix is None:
        raise KeyError(f"Unrecognized prefix in key '{key}'")

    # Extract subfolder name (everything after prefix + underscore)
    subfolder = key[len(prefix) + 1:]

    # Map to the actual folder name
    folder_name = feature_folders[prefix]

    # Build the target directory path
    output_dir = os.path.join(output_root, folder_name, subfolder)
    os.makedirs(output_dir, exist_ok=True)

    # Define output CSV path (same name as subfolder)
    csv_path = os.path.join(output_dir, f"{subfolder}.csv")

    # Write CSV without the index column
    df.to_csv(csv_path, index=False)
    print(f"Saved {key!r} to {csv_path}")

## BPI15A

### Load each df

In [ ]:
feature_folders = {
    'decl2': 'BPI15A_decl2_features',
    'mr_tr': 'BPI15A_mr_tr_features',
    'payload': 'BPI15A_payload_560925_features'
}

# Initialize dict for all DataFrames
dataframes = {}

# Remove all files in each subfolder except the one named <subfolder>.csv
for prefix, folder_name in feature_folders.items():
    base_dir = os.path.join(
        PROJECT_ROOT,
        '3.1_selected_features',
        'BPI15A',
        folder_name
    )
    # Loop over each subfolder (e.g. 'mr', 'dc_data', etc.)
    for subfolder in os.listdir(base_dir):
        subfolder_dir = os.path.join(base_dir, subfolder)
        if not os.path.isdir(subfolder_dir):
            continue

        # The only file we want to keep
        keep_name = f"{subfolder}.csv"

        # Delete everything else in this folder
        for fname in os.listdir(subfolder_dir):
            fpath = os.path.join(subfolder_dir, fname)
            if os.path.isfile(fpath) and fname != keep_name:
                os.remove(fpath)
                print(f"Removed unwanted file: {fpath}")

    # Find all CSV files one level down (e.g. bs_data/bs_data.csv, dc_data/dc_data.csv, etc.)
    csv_paths = glob.glob(os.path.join(base_dir, '*', '*.csv'))
    
    for path in csv_paths:
        # Extract the subfolder name, e.g. 'bs_data'
        subfolder = os.path.basename(os.path.dirname(path))
        # Create a prefixed key for the DataFrame dict
        key = f"{prefix}_{subfolder}"
        # Read the CSV
        df = pd.read_csv(path)
        dataframes[key] = df
        print(f"Loaded {key!r}: shape={df.shape}")

# Example access:
# dataframes['decl3_bs_data'], dataframes['mr_tr_dc_data'], dataframes['payload_baseline'], etc.


### Show unique values per df

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

In [ ]:
show_unique_counts('payload_IMPresseD')

### Strategy 1: Look for the optimal number of bins for variables with +50 unique values

In [ ]:
# threshold_unique = 50
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=10)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl2_bs_data':")
# for col, info in all_results['decl2_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Strategy 2: Hard coding 2/3 bins and a threshold of 10+ unique values

In [ ]:
# threshold_unique = 10
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl2_bs_data':")
# for col, info in all_results['decl2_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Strategy 3: Hard coding 2/3 bins and a threshold of 3+ unique values

In [ ]:
threshold_unique = 3
all_results = {}

for name, df in dataframes.items():
    # skip any dataset without the target
    if 'Label' not in df.columns:
        print(f"Skipping {name!r}: no 'Label' column")
        continue

    results = {}
    for col in df.columns:
        # you can also skip Label and Case_ID explicitly
        if col in ('Label','Case_ID'):
            continue
        if not pd.api.types.is_numeric_dtype(df[col]):
            continue
        if df[col].nunique() <= threshold_unique:
            continue

        best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
        results[col] = {'best_k': best_k, 'mi_by_k': scores}

    all_results[name] = results

In [ ]:
# Example: inspect bs_data results
print("Optimal bins for 'decl2_bs_data':")
for col, info in all_results['decl2_bs_data'].items():
    print(f" • {col}: {info['best_k']} bins")

In [ ]:
for name, results in all_results.items():
    df = dataframes[name]
    binned_from = []
    for col, info in results.items():
        k = info['best_k']
        # skip features where we couldn’t find a valid k
        if k is None:
            continue

        # apply equal-frequency bins with qcut
        binned_col = f"{col}_binned"
        df[binned_col] = pd.qcut(
            df[col],
            q=k,
            duplicates="drop"
        )
        binned_from.append(col)
        print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

    # drop all the originals at once (ignore any that might be missing)
    df = df.drop(columns=binned_from, errors='ignore')

    # save back if you want to overwrite
    dataframes[name] = df

### Show unique values per df after binning

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

In [ ]:
show_unique_counts('payload_IMPresseD')

### Export to 3.1_binned_logs

In [ ]:
# Define output root (parallel to '3_processed_logs')
output_root = os.path.join(PROJECT_ROOT, '3.2_binned_features', 'BPI15A')

# Save each DataFrame to its mirrored folder structure
for key, df in dataframes.items():
    # Determine which prefix this key uses
    prefix = None
    for p in feature_folders:
        if key.startswith(f"{p}_"):
            prefix = p
            break
    if prefix is None:
        raise KeyError(f"Unrecognized prefix in key '{key}'")

    # Extract subfolder name (everything after prefix + underscore)
    subfolder = key[len(prefix) + 1:]

    # Map to the actual folder name
    folder_name = feature_folders[prefix]

    # Build the target directory path
    output_dir = os.path.join(output_root, folder_name, subfolder)
    os.makedirs(output_dir, exist_ok=True)

    # Define output CSV path (same name as subfolder)
    csv_path = os.path.join(output_dir, f"{subfolder}.csv")

    # Write CSV without the index column
    df.to_csv(csv_path, index=False)
    print(f"Saved {key!r} to {csv_path}")

## Sepsis

### Load each df

In [ ]:
feature_folders = {
    'decl': 'sepsis_decl_features',
    'mr_tr': 'sepsis_mr_tr_features',
    'payload': 'sepsis_payload2_features'
}

# Initialize dict for all DataFrames
dataframes = {}

# Remove all files in each subfolder except the one named <subfolder>.csv
for prefix, folder_name in feature_folders.items():
    base_dir = os.path.join(
        PROJECT_ROOT,
        '3.1_selected_features',
        'sepsis',
        folder_name
    )
    # Loop over each subfolder (e.g. 'mr', 'dc_data', etc.)
    for subfolder in os.listdir(base_dir):
        subfolder_dir = os.path.join(base_dir, subfolder)
        if not os.path.isdir(subfolder_dir):
            continue

        # The only file we want to keep
        keep_name = f"{subfolder}.csv"

        # Delete everything else in this folder
        for fname in os.listdir(subfolder_dir):
            fpath = os.path.join(subfolder_dir, fname)
            if os.path.isfile(fpath) and fname != keep_name:
                os.remove(fpath)
                print(f"Removed unwanted file: {fpath}")
                
    # Find all CSV files one level down (e.g. bs_data/bs_data.csv, dc_data/dc_data.csv, etc.)
    csv_paths = glob.glob(os.path.join(base_dir, '*', '*.csv'))
    
    for path in csv_paths:
        # Extract the subfolder name, e.g. 'bs_data'
        subfolder = os.path.basename(os.path.dirname(path))
        # Create a prefixed key for the DataFrame dict
        key = f"{prefix}_{subfolder}"
        # Read the CSV
        df = pd.read_csv(path)
        dataframes[key] = df
        print(f"Loaded {key!r}: shape={df.shape}")

# Example access:
# dataframes['decl3_bs_data'], dataframes['mr_tr_dc_data'], dataframes['payload_baseline'], etc.


### Show unique values per df

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

In [ ]:
show_unique_counts('payload_IMPresseD')

### Strategy 1: Look for the optimal number of bins for variables with +50 unique values

In [ ]:
# threshold_unique = 50
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=10)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl_bs_data':")
# for col, info in all_results['decl_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Strategy 2: Hard coding 2/3 bins and a threshold of 10+ unique values

In [ ]:
# threshold_unique = 10
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect IMPresseD results
# print("Optimal bins for 'IMPresseD':")
# for col, info in all_results['decl_IMPresseD'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Strategy 3: Hard coding 2/3 bins and a threshold of 3+ unique values

In [ ]:
threshold_unique = 3
all_results = {}

for name, df in dataframes.items():
    # skip any dataset without the target
    if 'Label' not in df.columns:
        print(f"Skipping {name!r}: no 'Label' column")
        continue

    results = {}
    for col in df.columns:
        # you can also skip Label and Case_ID explicitly
        if col in ('Label','Case_ID'):
            continue
        if not pd.api.types.is_numeric_dtype(df[col]):
            continue
        if df[col].nunique() <= threshold_unique:
            continue

        best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
        results[col] = {'best_k': best_k, 'mi_by_k': scores}

    all_results[name] = results

In [ ]:
# # Example: inspect bs_data results
# print("Optimal bins for 'decl_bs_data':")
# for col, info in all_results['decl_bs_data'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
for name, results in all_results.items():
    df = dataframes[name]
    binned_from = []
    for col, info in results.items():
        k = info['best_k']
        # skip features where we couldn’t find a valid k
        if k is None:
            continue

        # apply equal-frequency bins with qcut
        binned_col = f"{col}_binned"
        df[binned_col] = pd.qcut(
            df[col],
            q=k,
            duplicates="drop"
        )
        binned_from.append(col)
        print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

    # drop all the originals at once (ignore any that might be missing)
    df = df.drop(columns=binned_from, errors='ignore')

    # save back if you want to overwrite
    dataframes[name] = df

### Show unique values per df after binning

In [ ]:
unique_counts_dict = {
    name: df.drop(columns=exclude, errors="ignore").nunique()
    for name, df in dataframes.items()
}

total_unique_dict = {
    name: counts.sum()
    for name, counts in unique_counts_dict.items()
}

In [ ]:
show_unique_counts('mr_tr_IMPresseD')

### Export to 3.1_binned_logs

In [ ]:
# Define output root (parallel to '3_processed_logs')
output_root = os.path.join(PROJECT_ROOT, '3.2_binned_features', 'sepsis')

# Save each DataFrame to its mirrored folder structure
for key, df in dataframes.items():
    # Determine which prefix this key uses
    prefix = None
    for p in feature_folders:
        if key.startswith(f"{p}_"):
            prefix = p
            break
    if prefix is None:
        raise KeyError(f"Unrecognized prefix in key '{key}'")

    # Extract subfolder name (everything after prefix + underscore)
    subfolder = key[len(prefix) + 1:]

    # Map to the actual folder name
    folder_name = feature_folders[prefix]

    # Build the target directory path
    output_dir = os.path.join(output_root, folder_name, subfolder)
    os.makedirs(output_dir, exist_ok=True)

    # Define output CSV path (same name as subfolder)
    csv_path = os.path.join(output_dir, f"{subfolder}.csv")

    # Write CSV without the index column
    df.to_csv(csv_path, index=False)
    print(f"Saved {key!r} to {csv_path}")

## DHL

### Load each df

In [ ]:
# feature_folders = {
#     'dhl': 'dhl_features',
# }

# # Initialize dict for all DataFrames
# dataframes = {}

# # Remove all files in each subfolder except the one named <subfolder>.csv
# for prefix, folder_name in feature_folders.items():
#     base_dir = os.path.join(
#         PROJECT_ROOT,
#         '3.1_selected_features',
#         'DHL',
#         folder_name
#     )
#     # Loop over each subfolder (e.g. 'mr', 'dc_data', etc.)
#     for subfolder in os.listdir(base_dir):
#         subfolder_dir = os.path.join(base_dir, subfolder)
#         if not os.path.isdir(subfolder_dir):
#             continue

#         # The only file we want to keep
#         keep_name = f"{subfolder}.csv"

#         # Delete everything else in this folder
#         for fname in os.listdir(subfolder_dir):
#             fpath = os.path.join(subfolder_dir, fname)
#             if os.path.isfile(fpath) and fname != keep_name:
#                 os.remove(fpath)
#                 print(f"Removed unwanted file: {fpath}")
                
#     # Find all CSV files one level down (e.g. bs_data/bs_data.csv, dc_data/dc_data.csv, etc.)
#     csv_paths = glob.glob(os.path.join(base_dir, '*', '*.csv'))
    
#     for path in csv_paths:
#         # Extract the subfolder name, e.g. 'bs_data'
#         subfolder = os.path.basename(os.path.dirname(path))
#         # Create a prefixed key for the DataFrame dict
#         key = f"{prefix}_{subfolder}"
#         # Read the CSV
#         df = pd.read_csv(path)
#         dataframes[key] = df
#         print(f"Loaded {key!r}: shape={df.shape}")

# # Example access:
# # dataframes['decl3_bs_data'], dataframes['mr_tr_dc_data'], dataframes['payload_baseline'], etc.


### Strategy 2: Hard coding 2/3 bins and a threshold of 10+ unique values

In [ ]:
# threshold_unique = 10
# all_results = {}

# for name, df in dataframes.items():
#     # skip any dataset without the target
#     if 'Label' not in df.columns:
#         print(f"Skipping {name!r}: no 'Label' column")
#         continue

#     results = {}
#     for col in df.columns:
#         # you can also skip Label and Case_ID explicitly
#         if col in ('Label','Case_ID'):
#             continue
#         if not pd.api.types.is_numeric_dtype(df[col]):
#             continue
#         if df[col].nunique() <= threshold_unique:
#             continue

#         best_k, scores = find_optimal_bins(df[col], df['Label'], k_min=2, k_max=3)
#         results[col] = {'best_k': best_k, 'mi_by_k': scores}

#     all_results[name] = results

In [ ]:
# # Example: inspect baseline results
# print("Optimal bins for 'baseline':")
# for col, info in all_results['dhl_baseline'].items():
#     print(f" • {col}: {info['best_k']} bins")

In [ ]:
# for name, results in all_results.items():
#     df = dataframes[name]
#     binned_from = []
#     for col, info in results.items():
#         k = info['best_k']
#         # skip features where we couldn’t find a valid k
#         if k is None:
#             continue

#         # apply equal-frequency bins with qcut
#         binned_col = f"{col}_binned"
#         df[binned_col] = pd.qcut(
#             df[col],
#             q=k,
#             duplicates="drop"
#         )
#         binned_from.append(col)
#         print(f"[{name}] {col!r} → {binned_col!r} with {k} bins")

#     # drop all the originals at once (ignore any that might be missing)
#     df = df.drop(columns=binned_from, errors='ignore')

#     # save back if you want to overwrite
#     dataframes[name] = df

### Export to 3.1_binned_logs

In [ ]:
# # Define output root (parallel to '3_processed_logs')
# output_root = os.path.join(PROJECT_ROOT, '3.2_binned_features', 'DHL')

# # Save each DataFrame to its mirrored folder structure
# for key, df in dataframes.items():
#     # Determine which prefix this key uses
#     prefix = None
#     for p in feature_folders:
#         if key.startswith(f"{p}_"):
#             prefix = p
#             break
#     if prefix is None:
#         raise KeyError(f"Unrecognized prefix in key '{key}'")

#     # Extract subfolder name (everything after prefix + underscore)
#     subfolder = key[len(prefix) + 1:]

#     # Map to the actual folder name
#     folder_name = feature_folders[prefix]

#     # Build the target directory path
#     output_dir = os.path.join(output_root, folder_name, subfolder)
#     os.makedirs(output_dir, exist_ok=True)

#     # Define output CSV path (same name as subfolder)
#     csv_path = os.path.join(output_dir, f"{subfolder}.csv")

#     # Write CSV without the index column
#     df.to_csv(csv_path, index=False)
#     print(f"Saved {key!r} to {csv_path}")

### Create overview of binning for report

#### Summarized overview

In [ ]:
# === Config ===
BINNED_BASE_DIR   = '3.2_binned_features'
ORIGINAL_BASE_DIR = '3.1_selected_features'  # pre-binning counterpart
summary_dir  = os.path.join(BINNED_BASE_DIR, 'short_summary')
os.makedirs(summary_dir, exist_ok=True)
summary_path = os.path.join(summary_dir, 'short_summary.csv')

# Name for the new binned-unique-results column (edit each run)
UNIQUE_COL_NAME = 'Unique values – strategy 2'  # e.g., 'Unique values – strategy 5'

# Encodings to ignore (case-insensitive exact folder names)
EXCLUDED_ENCODINGS = {'impressed', 'mr', 'mra', 'tr', 'tra'}

DROP_COLS = ['Case_ID', 'Label']
KEYS = ['Event Log', 'Labeling']

def _normalize_labeling(val: str):
    s = str(val).lower()
    if 'mr_tr' in s:
        return 'sequential'
    if 'decl' in s:
        return 'declare'
    if 'payload' in s:
        return 'payload'
    return val

def _sum_unique_from_folder(folder_path: str) -> int:
    """Sum unique counts across all CSVs in a folder (dropping Case_ID/Label)."""
    if not os.path.isdir(folder_path):
        return 0
    total_unique = 0
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(folder_path, fname))
        df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
        total_unique += df.nunique().sum()
    return total_unique

# === Load existing summary (if any), normalize, and coerce to (Event Log, Labeling) grain ===
if os.path.exists(summary_path):
    existing = pd.read_csv(summary_path)
    if not existing.empty and 'Labeling' in existing.columns:
        existing['Labeling'] = existing['Labeling'].apply(_normalize_labeling)
    # Drop legacy "Total Features" if present
    if 'Total Features' in existing.columns:
        existing = existing.drop(columns=['Total Features'])
    # If an older file still has 'Encoding', aggregate it away
    if 'Encoding' in existing.columns:
        numeric_cols = existing.select_dtypes(include='number').columns.tolist()
        existing = existing.groupby(KEYS, as_index=False)[numeric_cols].sum()
else:
    existing = pd.DataFrame()

# Build a lookup for already-known Original unique values so we don't recompute them
existing_orig_lookup = {}
if not existing.empty and 'Original unique values' in existing.columns:
    tmp = existing[KEYS + ['Original unique values']].dropna(subset=['Original unique values'])
    if not tmp.empty:
        # If duplicates exist, aggregate (sum) at labeling-grain to be safe
        tmp = tmp.groupby(KEYS, as_index=False)['Original unique values'].sum()
        existing_orig_lookup = {(r['Event Log'], r['Labeling']): r['Original unique values'] for _, r in tmp.iterrows()}

# === Build fresh binned unique counts per encoding (skipping excluded), then aggregate per labeling ===
records = []

for event_log in sorted(os.listdir(BINNED_BASE_DIR)):
    el_path = os.path.join(BINNED_BASE_DIR, event_log)
    if not os.path.isdir(el_path) or event_log == 'short_summary':
        continue

    label_strats = sorted(d for d in os.listdir(el_path) if os.path.isdir(os.path.join(el_path, d)))
    if not label_strats:
        continue

    for labeling in label_strats:
        strat_path_binned = os.path.join(el_path, labeling)

        for encoding in sorted(os.listdir(strat_path_binned)):
            if encoding.lower() in EXCLUDED_ENCODINGS:
                continue
            enc_path_binned = os.path.join(strat_path_binned, encoding)
            if not os.path.isdir(enc_path_binned):
                continue

            # Binned unique values for this encoding
            binned_unique = _sum_unique_from_folder(enc_path_binned)
            # Keep per-encoding; we'll aggregate to (Event Log, Labeling) below
            records.append({
                'Event Log': event_log,
                'Labeling': labeling,
                'Encoding': encoding,
                UNIQUE_COL_NAME: binned_unique,
            })

# Per-encoding -> DataFrame
if records:
    df_enc = pd.DataFrame(records).sort_values(['Event Log', 'Labeling', 'Encoding']).reset_index(drop=True)
else:
    df_enc = pd.DataFrame(columns=['Event Log', 'Labeling', 'Encoding', UNIQUE_COL_NAME])

# Normalize labeling
df_enc['Labeling'] = df_enc['Labeling'].apply(_normalize_labeling)

# Aggregate per (Event Log, Labeling) for the new strategy column
if not df_enc.empty:
    new_agg = df_enc.groupby(KEYS, as_index=False)[[UNIQUE_COL_NAME]].sum()
else:
    new_agg = pd.DataFrame(columns=KEYS + [UNIQUE_COL_NAME])

# Compute Original unique values ONLY for keys that don't already have it
orig_values = []
for _, row in new_agg.iterrows():
    key = (row['Event Log'], row['Labeling'])
    if key in existing_orig_lookup:
        orig_values.append(existing_orig_lookup[key])
    else:
        # Compute once per (Event Log, Labeling) by summing across all encodings (skipping excluded)
        strat_path_orig = os.path.join(ORIGINAL_BASE_DIR, row['Event Log'], row['Labeling'])
        total_orig_unique = 0
        if os.path.isdir(strat_path_orig):
            for enc in sorted(os.listdir(strat_path_orig)):
                if enc.lower() in EXCLUDED_ENCODINGS:
                    continue
                enc_path_orig = os.path.join(strat_path_orig, enc)
                if os.path.isdir(enc_path_orig):
                    total_orig_unique += _sum_unique_from_folder(enc_path_orig)
        orig_values.append(total_orig_unique)

new_agg['Original unique values'] = orig_values

# === Merge/update with existing (append/overwrite strategy column, keep existing Original unique values) ===
if existing.empty:
    updated = new_agg.copy()
else:
    ex = existing.set_index(KEYS)
    nw = new_agg.set_index(KEYS)

    # Ensure required columns exist in existing
    if 'Original unique values' not in ex.columns:
        ex['Original unique values'] = pd.NA
    if UNIQUE_COL_NAME not in ex.columns:
        ex[UNIQUE_COL_NAME] = pd.NA

    # Update the strategy column for matching rows
    common_idx = nw.index.intersection(ex.index)
    ex.loc[common_idx, UNIQUE_COL_NAME] = nw.loc[common_idx, UNIQUE_COL_NAME]

    # Only fill Original unique values where missing in existing
    need_orig_mask = ex.loc[common_idx, 'Original unique values'].isna()
    if need_orig_mask.any():
        idx_to_fill = need_orig_mask[need_orig_mask].index
        ex.loc[idx_to_fill, 'Original unique values'] = nw.loc[idx_to_fill, 'Original unique values']

    # Append any new rows
    missing_idx = nw.index.difference(ex.index)
    ex = pd.concat([ex, nw.loc[missing_idx]], axis=0)

    updated = ex.reset_index()

# Final ordering: Event Log, Labeling, Original unique values, then the rest (incl. the new strategy column)
front = ['Event Log', 'Labeling', 'Original unique values']
rest = [c for c in updated.columns if c not in front]
updated = updated[front + rest].sort_values(KEYS).reset_index(drop=True)

# Ensure legacy columns are gone
if 'Total Features' in updated.columns:
    updated = updated.drop(columns=['Total Features'])
if 'Encoding' in updated.columns:
    updated = updated.drop(columns=['Encoding'])

# Save
updated.to_csv(summary_path, index=False)
print(f"✅ Summary (per Labeling) updated at: {summary_path}")
print(f"🆕 Binned column written: {UNIQUE_COL_NAME}")
print("ℹ️ 'Original unique values' preserved/reused when already present.")
updated


#### Long overview

In [ ]:
import os
import pandas as pd

# === Config/paths ===
BINNED_BASE_DIR   = '3.2_binned_features'
ORIGINAL_BASE_DIR = '3.1_selected_features'
long_dir  = os.path.join(BINNED_BASE_DIR, 'long_overview')
os.makedirs(long_dir, exist_ok=True)
long_path = os.path.join(long_dir, 'long_overview.csv')

# Encodings to ignore (case-insensitive exact folder names)
EXCLUDED_ENCODINGS = {'impressed', 'mr', 'mra', 'tr', 'tra'}

DROP_COLS = ['Case_ID', 'Label']
KEYS = ['Event Log', 'Labeling', 'Encoding']

# === Reuse UNIQUE_COL_NAME from previous cell; try to infer if missing ===
try:
    UNIQUE_COL_NAME
except NameError:
    ss_path = os.path.join(BINNED_BASE_DIR, 'short_summary', 'short_summary.csv')
    if os.path.exists(ss_path):
        _ss = pd.read_csv(ss_path)
        cand = [c for c in _ss.columns if isinstance(c, str) and c.lower().startswith('unique values')]
        UNIQUE_COL_NAME = cand[-1] if cand else 'Unique values – strategy'
    else:
        UNIQUE_COL_NAME = 'Unique values – strategy'

def _normalize_labeling(val: str):
    s = str(val).lower()
    if 'mr_tr' in s:
        return 'sequential'
    if 'decl' in s:
        return 'declare'
    if 'payload' in s:
        return 'payload'
    return val

def _sum_unique_from_folder(folder_path: str) -> int:
    """Sum unique counts across all CSVs in a folder (dropping Case_ID/Label)."""
    if not os.path.isdir(folder_path):
        return 0
    total_unique = 0
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(folder_path, fname))
        df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
        total_unique += df.nunique().sum()
    return total_unique

# === Load existing long_overview (if any) ===
if os.path.exists(long_path):
    existing = pd.read_csv(long_path)
    if not existing.empty and 'Labeling' in existing.columns:
        existing['Labeling'] = existing['Labeling'].apply(_normalize_labeling)
    # Drop legacy columns if present
    for col in ('Total Features',):
        if col in existing.columns:
            existing = existing.drop(columns=[col])
else:
    existing = pd.DataFrame()

# Build lookup to avoid recomputing "Original unique values"
existing_orig_lookup = {}
if not existing.empty and 'Original unique values' in existing.columns:
    tmp = existing[KEYS + ['Original unique values']].dropna(subset=['Original unique values'])
    if not tmp.empty:
        # If duplicates exist, take the last non-null (or sum). We'll take last to preserve prior values.
        tmp = tmp.drop_duplicates(subset=KEYS, keep='last')
        existing_orig_lookup = {
            (r['Event Log'], r['Labeling'], r['Encoding']): r['Original unique values']
            for _, r in tmp.iterrows()
        }

# === Build fresh per-encoding records ===
records = []
for event_log in sorted(os.listdir(BINNED_BASE_DIR)):
    if event_log in ('short_summary', 'long_overview'):
        continue
    el_path = os.path.join(BINNED_BASE_DIR, event_log)
    if not os.path.isdir(el_path):
        continue

    label_strats = sorted(d for d in os.listdir(el_path) if os.path.isdir(os.path.join(el_path, d)))
    if not label_strats:
        continue

    for labeling in label_strats:
        # use non-normalized label for pathing; normalized for the row value
        labeling_path_binned = os.path.join(el_path, labeling)
        labeling_path_orig   = os.path.join(ORIGINAL_BASE_DIR, event_log, labeling)
        labeling_norm = _normalize_labeling(labeling)

        for encoding in sorted(os.listdir(labeling_path_binned)):
            if encoding.lower() in EXCLUDED_ENCODINGS:
                continue
            enc_path_binned = os.path.join(labeling_path_binned, encoding)
            if not os.path.isdir(enc_path_binned):
                continue

            # binned unique values for this encoding
            binned_unique = _sum_unique_from_folder(enc_path_binned)

            # original unique values (reuse existing if available)
            key = (event_log, labeling_norm, encoding)
            if key in existing_orig_lookup:
                orig_unique = existing_orig_lookup[key]
            else:
                enc_path_orig = os.path.join(labeling_path_orig, encoding)
                orig_unique = _sum_unique_from_folder(enc_path_orig)

            records.append({
                'Event Log': event_log,
                'Labeling': labeling_norm,
                'Encoding': encoding,
                'Original unique values': orig_unique,
                UNIQUE_COL_NAME: binned_unique,
            })

# New dataframe
if records:
    new_df = pd.DataFrame(records).sort_values(KEYS).reset_index(drop=True)
else:
    new_df = pd.DataFrame(columns=KEYS + ['Original unique values', UNIQUE_COL_NAME])

# === Merge/update with existing ===
if existing.empty:
    updated = new_df.copy()
else:
    ex = existing.set_index(KEYS)
    nw = new_df.set_index(KEYS)

    # Ensure columns exist
    if 'Original unique values' not in ex.columns:
        ex['Original unique values'] = pd.NA
    if UNIQUE_COL_NAME not in ex.columns:
        ex[UNIQUE_COL_NAME] = pd.NA

    common_idx = nw.index.intersection(ex.index)

    # Update strategy column
    ex.loc[common_idx, UNIQUE_COL_NAME] = nw.loc[common_idx, UNIQUE_COL_NAME]

    # Only fill original unique where missing
    need_orig_mask = ex.loc[common_idx, 'Original unique values'].isna()
    if need_orig_mask.any():
        idx_to_fill = need_orig_mask[need_orig_mask].index
        ex.loc[idx_to_fill, 'Original unique values'] = nw.loc[idx_to_fill, 'Original unique values']

    # Append new rows
    missing_idx = nw.index.difference(ex.index)
    ex = pd.concat([ex, nw.loc[missing_idx]], axis=0)

    updated = ex.reset_index()

# Order columns: Event Log, Labeling, Encoding, Original unique values, then the rest
front = ['Event Log', 'Labeling', 'Encoding', 'Original unique values']
rest  = [c for c in updated.columns if c not in front]
updated = updated[front + rest].sort_values(KEYS).reset_index(drop=True)

# Save
updated.to_csv(long_path, index=False)
print(f"✅ Long overview (per Encoding) updated at: {long_path}")
print(f"🆕 Binned column written: {UNIQUE_COL_NAME}")
print("ℹ️ 'Original unique values' preserved/reused when already present.")
updated


In [ ]:
import os
import re
import pandas as pd

# --- helper for TeX-safe cell values ---
def fmt_rule(x):
    return r'\detokenize{' + str(x) + '}'

# export targets
fp_short_tex = os.path.join('3.2_binned_features', 'short_summary', 'short_summary.tex')
fp_long_tex  = os.path.join('3.2_binned_features', 'long_overview',  'long_summary.tex')

# source CSVs
fp_short_csv = os.path.join('3.2_binned_features', 'short_summary', 'short_summary.csv')
fp_long_csv  = os.path.join('3.2_binned_features', 'long_overview',  'long_overview.csv')

def _colalign(df: pd.DataFrame) -> str:
    return ''.join('r' if pd.api.types.is_numeric_dtype(df[c]) else 'l' for c in df.columns)

def _to_integers(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    num_cols = out.select_dtypes(include='number').columns
    for c in num_cols:
        out[c] = pd.to_numeric(out[c], errors='coerce').round(0).astype('Int64')
    return out

def _reorder_unique_value_strategy_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find columns named like 'Unique values – strategy N' (N=1,2,3), sort by N,
    and place them back in that order where the first one originally appeared.
    Works with hyphen '-' or en dash '–', case-insensitive.
    """
    cols = list(df.columns)
    # capture (n, colname) for matching columns
    pat = re.compile(r'^\s*Unique\s+values\s*[–-]\s*strategy\s*(\d+)\s*$', re.IGNORECASE)
    matches = [(int(pat.match(str(c)).group(1)), c) for c in cols if pat.match(str(c))]
    if not matches:
        return df  # nothing to do

    # desired order by numeric suffix
    matches_sorted = sorted(matches, key=lambda t: t[0])
    uv_names_sorted = [c for _, c in matches_sorted]

    # where to reinsert (position of the first of these columns)
    first_pos = min(cols.index(c) for _, c in matches)
    # remove them from the current order
    base = [c for c in cols if c not in [name for _, name in matches]]
    # reinsert in sorted order
    base[first_pos:first_pos] = uv_names_sorted
    return df[base]

def _df_to_longtable_tex(df: pd.DataFrame, formatters: dict | None) -> str:
    tex = df.to_latex(
        index=False,
        escape=False,
        longtable=True,
        multicolumn=False,
        column_format=_colalign(df),
        na_rep='',
        formatters=formatters
    )
    return tex

def _save_tex(df: pd.DataFrame, out_path: str, formatters: dict | None):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, 'w', encoding='utf-8') as f:
        f.write(_df_to_longtable_tex(df, formatters))

# ---- SHORT SUMMARY ----
if os.path.exists(fp_short_csv):
    short_df = pd.read_csv(fp_short_csv)
    short_df = _reorder_unique_value_strategy_cols(short_df)
    short_df = _to_integers(short_df)
    all_fmt = {col: fmt_rule for col in short_df.columns}
    _save_tex(short_df, fp_short_tex, all_fmt)
    print(f"✅ Wrote short_summary TeX → {fp_short_tex}")
else:
    print(f"⚠️ Not found: {fp_short_csv}")

# ---- LONG OVERVIEW ----
if os.path.exists(fp_long_csv):
    long_df = pd.read_csv(fp_long_csv)
    long_df = _reorder_unique_value_strategy_cols(long_df)
    long_df = _to_integers(long_df)
    enc_fmt = {'Encoding': fmt_rule} if 'Encoding' in long_df.columns else None
    _save_tex(long_df, fp_long_tex, enc_fmt)
    print(f"✅ Wrote long_overview TeX → {fp_long_tex}")
else:
    print(f"⚠️ Not found: {fp_long_csv}")


### Checking unique values before binning

In [ ]:
# Aggregate unique-value counts PER ENCODING, excluding some encodings,
# and skipping 'Case_ID' and 'Label' columns from the counts.

from pathlib import Path
import pandas as pd

base_dir = Path("3.1_selected_features")
exclude_encodings = {"impressed", "mr", "mra", "tr", "tra", "dhl"}  # case-insensitive compare
exclude_cols = {"case_id", "label"}  # columns to skip (case-insensitive)

rows = []
missing = []

for log_dir in sorted([p for p in base_dir.iterdir() if p.is_dir()]):
    log = log_dir.name
    for lab_dir in sorted([p for p in log_dir.iterdir() if p.is_dir()]):
        labeling = lab_dir.name
        for enc_dir in sorted([p for p in lab_dir.iterdir() if p.is_dir()]):
            encoding = enc_dir.name
            if encoding.lower() in exclude_encodings:
                continue

            csv_files = sorted(enc_dir.glob("*.csv"))
            if not csv_files:
                missing.append((log, labeling, encoding, "⚠️ no CSV found"))
                continue

            for csv_fp in csv_files:
                try:
                    df = pd.read_csv(csv_fp, low_memory=False)
                except Exception as e:
                    missing.append((log, labeling, encoding, f"❌ failed to read: {csv_fp.name} ({e})"))
                    continue

                per_col_counts = []
                for col in df.columns:
                    if col.strip().lower() in exclude_cols:
                        continue
                    try:
                        per_col_counts.append(df[col].nunique(dropna=False))
                    except Exception:
                        per_col_counts.append(df[col].astype(str).nunique(dropna=False))

                total_unique = int(sum(int(x) for x in per_col_counts)) if per_col_counts else 0
                rows.append({
                    "Log": log,
                    "Labeling": labeling,
                    "Encoding": encoding,
                    "Columns counted": int(len(per_col_counts)),
                    "Unique values (total across columns)": total_unique,
                    "Source file": csv_fp.name,
                })

# Build aggregated result
agg_per_encoding = pd.DataFrame(
    rows,
    columns=["Log", "Labeling", "Encoding", "Columns counted", "Unique values (total across columns)", "Source file"]
)

# Collapse to single row per Log/Labeling/Encoding if multiple CSVs exist
if not agg_per_encoding.empty:
    agg_per_encoding = (
        agg_per_encoding
        .groupby(["Log", "Labeling", "Encoding"], as_index=False)
        .agg({
            "Columns counted": "sum",
            "Unique values (total across columns)": "sum",
        })
        .sort_values(["Log", "Labeling", "Encoding"], kind="stable")
        .reset_index(drop=True)
    )

print(f"✅ Encodings summarized: {len(agg_per_encoding):,}")
if missing:
    print("Notes/warnings:")
    for item in missing:
        print("  -", " / ".join(map(str, item)))

# Optional: save to disk
# agg_per_encoding.to_csv("3.1_selected_features_unique_counts_per_encoding.csv", index=False)

agg_per_encoding


In [ ]:
# # Further aggregate by LABELING (across encodings within each log/labeling)

# # If you ran the previous cell, `agg_per_encoding` already exists.
# # Otherwise, you can run that first or wrap it here.

# if 'agg_per_encoding' not in globals() or agg_per_encoding.empty:
#     raise RuntimeError("agg_per_encoding not found or empty. Run the previous cell first.")

# # How many encodings contribute to each (Log, Labeling) group?
# # (each row in agg_per_encoding is a single encoding after the prior collapse)
# agg_per_labeling = (
#     agg_per_encoding
#     .groupby(["Log", "Labeling"], as_index=False)
#     .agg(
#         Encodings_count=("Encoding", "nunique"),
#         Columns_counted_total=("Columns counted", "sum"),
#         Unique_values_total=("Unique values (total across columns)", "sum"),
#     )
# )

# # Optional derived metrics
# agg_per_labeling["Avg columns per encoding"] = (
#     agg_per_labeling["Columns_counted_total"] / agg_per_labeling["Encodings_count"]
# ).round(2)

# agg_per_labeling["Avg unique-values per encoding"] = (
#     agg_per_labeling["Unique_values_total"] / agg_per_labeling["Encodings_count"]
# ).round(2)

# # Sort for readability
# agg_per_labeling = agg_per_labeling.sort_values(["Log", "Labeling"], kind="stable").reset_index(drop=True)

# print(f"✅ Labeling groups summarized: {len(agg_per_labeling):,}")

# # Optional: save
# # agg_per_labeling.to_csv("3.1_selected_features_unique_counts_per_labeling.csv", index=False)

# agg_per_labeling
